# Ask about Video clip with Gemini 1.0 Pro Vision on Vertex AI

In [ ]:
!pip install --upgrade google-cloud-aiplatform

In [ ]:
from IPython.display import Markdown, display

## Authentication to Vertex AI with `gcloud`

In [ ]:
!gcloud auth application-default login

# or do the same thing without interrupting prompt
#
# export GOOGLE_APPLICATION_CREDENTIALS="/path/to/your/service_account_key.json"
# gcloud auth application-default login --client-id-file=/path/to/your/service_account_key.json

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=y5WPnVc0yhvIlt4n0fu87xzoPybWki&prompt=consent&token_usage=remote&access_type=offline&code_challenge=KB3CebOE1LzZOb0jHlY6xfMLvFYdyrb7Uv2nAnD9NEA&code_challenge_method=S256

Enter authorization code: 4/0AeaYSHAZqCL-B3-yRlALC9TU5yq9EFFZyivdJ6laefam31EDueOcnfyNuUsBx8zrInUbgA

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Cannot find a quota project to add to ADC. You might receive a "quota exceeded"

## Setup GCP Project and Location

In [ ]:
GCP_PROJECT_ID="gde-prj"
GCP_PROJECT_LOCATION="us-central1"

## Define Gemini call function

In [ ]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, GenerationResponse, GenerationConfig

def initi_vertexai(project_id: str, location: str) -> None:
    vertexai.init(project=project_id, location=location)

def ask_gemini(
    prompt: str=None, gcs: str=None, base64_encoded: bytes=None, stream: bool=False, generation_config: dict=None
) -> GenerationResponse:
    if gcs is None and base64_encoded is None:
        raise ValueError("Either a GCS bucket path or base64_encoded string of the video must be provided")

    if gcs is not None and base64_encoded is not None:
        raise ValueError("Only one of gcs or base64_encoded must be provided")

    if gcs is not None:
        video = Part.from_uri(gcs, mime_type="video/mp4")
    else:
        video = Part.from_data(data=base64.b64decode(base64_encoded), mime_type="video/mp4")

    if prompt is None:
        prompt = "What is in the video?"

    if generation_config is None:
        generation_config = GenerationConfig(
            max_output_tokens=2048,
            temperature=0.4,
            top_p=1,
            top_k=32
        )

    vision_model = GenerativeModel("gemini-1.0-pro-vision")
    return vision_model.generate_content(
        [video, prompt],
        generation_config=generation_config, stream=stream
    )

## Define base64 encoding function

In [ ]:
def get_base64_encode(file_path):
    with open(file_path, 'rb') as f:
        data = f.read()

    return base64.b64encode(data)

In [ ]:
sample1 = get_base64_encode("sample1.mp4")
sample2 = get_base64_encode("sample2.mp4")
sample3 = get_base64_encode("sample3.mp4")
sample4 = get_base64_encode("sample4.mp4")

## Define common prompt

In [ ]:
prompt = """
Describe a character appeared in the video as much detailed as possible.
Use your analytical skill and imagination to generate the response.
Come up with a fun name for the character as well.

Your response should be filled in the valid JSON format as below.

{
    "human": bool,
    "name": text,
    "mood": text,
    "internal personalities": [text],
    "external personalities": [text],
    "description": text
}
"""

## Let's ask!

### on Sample1

In [ ]:
initi_vertexai(GCP_PROJECT_ID, GCP_PROJECT_LOCATION)
try:
    response = ask_gemini(
        prompt=prompt,
        base64_encoded=sample1
    )
    display(Markdown(response.text))
except Exception as e:
    print(f"something went wrong {e}")

 ```json
{
  "human": true,
  "name": "Red Beanie Girl",
  "mood": "happy",
  "internal personalities": [
    "carefree",
    "spontaneous",
    "乐观"
  ],
  "external personalities": [
    "friendly",
    "approachable",
    "easygoing"
  ],
  "description": "Red Beanie Girl is a young woman who is full of life and loves to have fun. She is always up for an adventure and is always looking for new things to experience. She is a great friend and is always there for those she cares about."
}
```

### on Sample2

In [ ]:
initi_vertexai(GCP_PROJECT_ID, GCP_PROJECT_LOCATION)
try:
    response = ask_gemini(
        prompt=prompt,
        base64_encoded=sample2
    )
    display(Markdown(response.text))
except Exception as e:
    print(f"something went wrong {e}")

 ```json
{
    "human": false,
    "name": "Chibi",
    "mood": "curious",
    "internal personalities": ["cuddly", "mischevious"],
    "external personalities": ["cute", "furry"],
    "description": "Chibi is a small, furry creature with big eyes and a long tail. He is brown and has two horns on his head. He is very curious and loves to play. He is also very friendly and loves to cuddle."
}
```

### on Sample3

In [ ]:
initi_vertexai(GCP_PROJECT_ID, GCP_PROJECT_LOCATION)
try:
    response = ask_gemini(
        prompt=prompt,
        base64_encoded=sample3
    )
    display(Markdown(response.text))
except Exception as e:
    print(f"something went wrong {e}")

 ```json
{
  "human": true,
  "name": "Ripley",
  "mood": "calm",
  "internal personalities": [
    "introverted",
    "shy",
    "reserved"
  ],
  "external personalities": [
    "mysterious",
    "distant",
    "unapproachable"
  ],
  "description": "Ripley is a young woman who is trying to find her place in the world. She is a bit of a loner, but she is also very kind and caring. She is always looking for new experiences, and she is always up for a challenge."
}
```

### on Sample4

In [ ]:
initi_vertexai(GCP_PROJECT_ID, GCP_PROJECT_LOCATION)
try:
    response = ask_gemini(
        prompt=prompt,
        base64_encoded=sample4
    )
    display(Markdown(response.text))
except Exception as e:
    print(f"something went wrong {e}")

 ```json
{
  "human": true,
  "name": "Graham",
  "mood": "calm",
  "internal personalities": [
    "the artist",
    "the thinker",
    "the dreamer"
  ],
  "external personalities": [
    "the charmer",
    "the wit",
    "the raconteur"
  ],
  "description": "Graham is a retired art professor who lives in a small town in the south of France. He is a kind and gentle man who enjoys spending time with his family and friends. He is also a talented artist and enjoys painting in his spare time."
}
```